In [ ]:
from bs4 import BeautifulSoup
import mysql.connector
import pandas as pd 
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import tiktoken
import nltk
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
import json

set(stopwords.words('english'))
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
!pip show openai
!pip install openai==1.55.3

Name: openai
Version: 1.55.3
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: C:\Users\User\school\ISM v.1 Docker\.venv\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Write API key here
from openai import OpenAI
client = OpenAI(api_key= "Enter API Key Here")
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
import glob
import os

def get_xml_filenames(folder_path):
    return [os.path.basename(file) for file in glob.glob(os.path.join(folder_path, "*.xml"))]

folder_path = "../tei_output" 
xml_files = get_xml_filenames(folder_path)

In [4]:
def ngram(text, n):
    list_ngrams = []
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    ngram_text = ngrams(text.split(), n)
    for grams in ngram_text:
        list_ngrams.append(grams[0])

    return list_ngrams

In [5]:
def generate_embedding(text: str) -> list[float]:
	resp = client.embeddings.create(model=EMBEDDING_MODEL, input=text,)

	return resp.data[0].embedding

In [6]:
def split_list_by_token_limit(text_list, max_tokens=5000, model="text-embedding-ada-002"):
    encoding = tiktoken.encoding_for_model(model)
    
    chunks = []
    current_chunk = []
    current_tokens = 0

    for text in text_list:
        token_count = len(encoding.encode(text))  

        if token_count > max_tokens:
            sub_chunks = [text[i:i+max_tokens] for i in range(0, len(text), max_tokens)]
            for sub in sub_chunks:
                if current_tokens + len(encoding.encode(sub)) > max_tokens:
                    chunks.append(" ".join(current_chunk))
                    current_chunk = []
                    current_tokens = 0
                current_chunk.append(sub)
                current_tokens += len(encoding.encode(sub))
            continue 

        if current_tokens + token_count > max_tokens:
            chunks.append(" ".join(current_chunk)) 
            current_chunk = []
            current_tokens = 0
        
        current_chunk.append(text)
        current_tokens += token_count
        
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [7]:
def chunk_text(text, max_tokens=5000, model="text-embedding-ada-002"):

    encoding = tiktoken.encoding_for_model(model) 
    words = text.split() 
    chunks = []
    current_chunk = []
    current_tokens = 0

    for word in words:
        word_token_count = len(encoding.encode(word))

        if current_tokens + word_token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0

        current_chunk.append(word)
        current_tokens += word_token_count

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [13]:
def filter_ngrams(ngram_list, remove_count=1):
    ngram_freq = Counter(ngram_list)

    sorted_ngrams = ngram_freq.most_common()

    filtered_ngrams = sorted_ngrams[remove_count:]

    return [ngram for ngram, _ in filtered_ngrams]

In [10]:
conn = mysql.connector.connect(user='root',  
        password='',  
        host='localhost',
        database='ism') 

cursor = conn.cursor()

In [15]:
for count, file_name in enumerate(xml_files):
    with open(folder_path + "/"+ file_name, "r", encoding="utf-8") as file:
        xml_content = file.read()

    soup = BeautifulSoup(xml_content, "xml")
    for ref in soup.find_all("ref"):
        ref.unwrap()
    desc_text = soup.fileDesc.get_text(separator="\n", strip=True)
    title_text = soup.title.get_text(separator="\n", strip=True)
    abstract_text = soup.abstract.get_text(separator="\n", strip=True)
    body_text = soup.body.get_text(separator="\n", strip=True)

    abstract_embed = generate_embedding(abstract_text)
    abstract_bigrams = filter_ngrams(ngram(title_text+abstract_text,2), remove_count=1)
    abstract_bigram_embed = generate_embedding(str(abstract_bigrams))

    chunk_body= chunk_text(body_text)
    for i, chunk in enumerate(chunk_body):
        chunk_content = chunk
        chunk_embed = generate_embedding(str(chunk))

        combined_text = title_text + " " + abstract_text + " " + chunk_content
        list_bigrams = filter_ngrams(ngram(combined_text,2), remove_count=1)
        chunks = split_list_by_token_limit(list_bigrams)

        for i, chunk in enumerate(chunks):
            insert_query = """
            INSERT INTO data (file_name, title, description, abstract, abstract_embed, abstract_bigrams, abstract_bigram_embed, content, content_embed, content_bigrams, content_bigram_embed)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                str(file_name),
                title_text,
                desc_text,
                abstract_text,
                json.dumps(abstract_embed), 
                str(abstract_bigrams),
                json.dumps(abstract_bigram_embed),
                chunk_content,
                json.dumps(chunk_embed),
                str(chunk),
                json.dumps(generate_embedding(str(chunk))),
            )

    cursor.execute(insert_query, values)
    conn.commit()
    print( print(f"\rProcessing item {count+1}/{len(xml_files)}", end="", flush=True))

cursor.close()
conn.close()

print("Data successfully inserted into MySQL!")

Processing item 1/1850None
Processing item 2/1850None
Processing item 3/1850None
Processing item 4/1850None
Processing item 5/1850None
Processing item 6/1850None
Processing item 7/1850None
Processing item 8/1850None
Processing item 9/1850None
Processing item 10/1850None
Processing item 11/1850None
Processing item 12/1850None
Processing item 13/1850None
Processing item 14/1850None
Processing item 15/1850None
Processing item 16/1850None
Processing item 17/1850None
Processing item 18/1850None
Processing item 19/1850None
Processing item 20/1850None
Processing item 21/1850None
Processing item 22/1850None
Processing item 23/1850None
Processing item 24/1850None
Processing item 25/1850None
Processing item 26/1850None
Processing item 27/1850None
Processing item 28/1850None
Processing item 29/1850None
Processing item 30/1850None
Processing item 31/1850None
Processing item 32/1850None
Processing item 33/1850None
Processing item 34/1850None
Processing item 35/1850None
Processing item 36/1850None
P

In [ ]:
conn = mysql.connector.connect(user='root',  
        password='',  
        host='localhost',
        database='ism') 

query = "SELECT * FROM data"  
df = pd.read_sql(query, conn)
conn.close()
print(df)

C:\Users\User\AppData\Local\Temp\ipykernel_33224\2023882820.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


        id                                          file_name  \
0        1  2004 - Kruth et al. - Selective laser melting ...   
1        2  2004 - Shiomi et al. - Residual Stress within ...   
2        3  2005 - Kruth et al. - Binding mechanisms in se...   
3        4  2006 - Osakada and Shiomi - Flexible manufactu...   
4        5  2006 - Rombouts et al. - Fundamentals of Selec...   
...    ...                                                ...   
1845  1846  Voigt et al. - Edge effects in radial porosity...   
1846  1847  Yakout et al. - Selective laser melting of sof...   
1847  1848  Yang et al. - ANALYZING REMELTING CONDITIONS B...   
1848  1849  Ye et al. - Absorptivity measurements and scal...   
1849  1850  Yu - Potential Solution to Meet Growing Demand...   

                                                  title  \
0          Selective laser melting of iron-based powder   
1     Residual Stress within Metallic Model Made by ...   
2     Binding mechanisms in selective lase

: 